# Weeks 3-4: Markov Chains

**Objective:** Understand the Markov property, represent a system using a transition probability matrix, simulate its behavior, and calculate its long-run (stationary) distribution.

## Step 1: Build Intuition

Imagine you want to predict tomorrow's weather. You could look at the weather for the past month, but a simpler, often effective, strategy is to just look at today's weather. If it's sunny today, there's a certain chance it will be sunny tomorrow, and a certain chance it will be rainy. The key idea is that **the future depends only on the present, not the past**.

This "memoryless" property is the heart of a Markov Chain. It doesn't matter if it was rainy for the last 10 days; if today is sunny, the prediction for tomorrow is based *only* on the fact that today is sunny.

**Examples:**
- A board game where your next position depends only on your current square and a dice roll.
- A customer's brand loyalty: their next purchase depends on the brand they bought last, not their entire purchase history.
- The random walk from last week is a perfect example of a Markov Chain!

## Step 2: Understand the Core Idea

To define a Markov Chain, we need three things:

1.  **State Space:** A set of all possible conditions or states the system can be in. For our weather example, the state space is `S = {Sunny, Rainy}`.
2.  **Transition Probabilities:** The probabilities of moving from one state to another in a single time step. For example, the probability of moving from `Sunny` to `Rainy`.
3.  **Markov Property:** The probability of moving to the next state `j` depends *only* on the current state `i`. It is independent of all past states.

## Step 3: Learn the Definitions and Formulas

**Definition: Markov Chain**
A discrete-time stochastic process \({X_n, n \ge 0}\) is a Markov chain if it satisfies the Markov property:
$$ P(X_{n+1} = j | X_n = i, X_{n-1} = i_{n-1}, ..., X_0 = i_0) = P(X_{n+1} = j | X_n = i) $$
for all time steps \(n\) and all states \(i, j, i_0, ...\).

--- 

**Definition: Transition Probability Matrix (TPM)**
We can organize the transition probabilities into a matrix \(P\), where the entry \(P_{ij}\) is the probability of moving from state \(i\) to state \(j\) in one step.
$$ P_{ij} = P(X_{n+1} = j | X_n = i) $$

For our weather example, let's define the states as `0 = Sunny` and `1 = Rainy`. The TPM could be:

$$ P = \begin{pmatrix} P(S \to S) & P(S \to R) \\ P(R \to S) & P(R \to R) \end{pmatrix} = \begin{pmatrix} 0.9 & 0.1 \\ 0.5 & 0.5 \end{pmatrix} $$

**Properties of a TPM:**
1. All entries must be non-negative: \(P_{ij} \ge 0\).
2. The sum of each row must be 1 (from any state, you must transition to *some* other state): \(\sum_j P_{ij} = 1\) for all \(i\).

--- 

**Definition: Stationary Distribution (or Equilibrium)**
A stationary distribution is a probability distribution \(\pi\) over the states that does not change over time. If the system reaches this distribution, it will stay there. It is a row vector \(\pi\) that satisfies:
$$ \pi P = \pi $$
This means \(\pi\) is a **left eigenvector** of the matrix \(P\) with an eigenvalue of 1.

## Step 4: Apply and Practice

Let's use Python to model the weather example. We'll define the TPM, simulate the weather over a month, and then calculate the stationary distribution to find the long-term weather forecast.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8-whitegrid')

### Part A: Defining the System and Simulating

First, let's define our states and the Transition Probability Matrix (TPM).

In [ ]:
# State space: 0 for Sunny, 1 for Rainy
states = ['Sunny', 'Rainy']

# Transition Probability Matrix (TPM)
# P[i, j] is the probability of transitioning from state i to state j
P = np.array([
    [0.9, 0.1],  # P(Sunny -> Sunny), P(Sunny -> Rainy)
    [0.5, 0.5]   # P(Rainy -> Sunny), P(Rainy -> Rainy)
])

print("Transition Matrix P:\n", P)

Now, let's simulate the weather for 30 days, assuming it starts as Sunny.

In [ ]:
def simulate_weather(tpm, num_days, start_state=0):
    """Simulates a path of the Markov chain."""
    num_states = tpm.shape[0]
    path = [start_state]
    current_state = start_state
    
    for _ in range(num_days - 1):
        # Get the transition probabilities for the current state
        transition_probs = tpm[current_state, :]
        # Choose the next state based on these probabilities
        next_state = np.random.choice(np.arange(num_states), p=transition_probs)
        path.append(next_state)
        current_state = next_state
        
    return path

# Simulate for 30 days, starting with a Sunny day (state 0)
N_DAYS = 30
weather_path = simulate_weather(P, N_DAYS, start_state=0)

# Convert numeric path to state names for readability
weather_forecast = [states[i] for i in weather_path]

print(f"Simulated weather for {N_DAYS} days:")
print(" -> ".join(weather_forecast))

### Part B: Multi-Step Transitions

What is the probability of the weather being Rainy 3 days from now, if it is Sunny today? This is given by the (0, 1) entry of the matrix \(P^3\).

In [ ]:
# To find the probabilities after n steps, we compute P^n
# Let's find the 3-day transition matrix
P_3_days = np.linalg.matrix_power(P, 3)

print("Transition Matrix after 3 days (P^3):\n", P_3_days)

prob_sunny_to_rainy_3_days = P_3_days[0, 1]
print(f"\nProbability of it being Rainy in 3 days, given it's Sunny today: {prob_sunny_to_rainy_3_days:.4f}")

### Part C: Stationary Distribution

What is the long-term probability of a day being Sunny or Rainy? We need to find the stationary distribution \(\pi\) such that \(\pi P = \pi\).

This is equivalent to finding the left eigenvector of \(P\) corresponding to the eigenvalue 1. We can rewrite \(\pi P = \pi\) as \(\pi (P - I) = 0\), where \(I\) is the identity matrix. In NumPy, we find the *right* eigenvectors, so we work with the transpose: \((P^T - I^T) \pi^T = 0\).

In [ ]:
def find_stationary_distribution(tpm):
    """Calculates the stationary distribution of a Markov chain."""
    # We need to solve pi * P = pi, which is equivalent to pi * (P - I) = 0
    # This means pi is the left eigenvector of P with eigenvalue 1.
    # In numpy, we find right eigenvectors, so we work with P.T
    eigenvalues, eigenvectors = np.linalg.eig(tpm.T)
    
    # Find the eigenvector corresponding to the eigenvalue 1
    # Note: due to floating point, we check for closeness to 1
    stationary_vector = eigenvectors[:, np.isclose(eigenvalues, 1)]
    
    # The eigenvector is complex by default, so take the real part
    # and normalize it to sum to 1
    stationary_dist = stationary_vector.real
    stationary_dist /= stationary_dist.sum()
    
    return stationary_dist.flatten()

pi = find_stationary_distribution(P)

print(f"The stationary distribution is:")
print(f"- Probability of Sunny: {pi[0]:.4f}")
print(f"- Probability of Rainy: {pi[1]:.4f}")

# Verification: Check if pi * P = pi
print("\nVerification (pi * P):", pi @ P)
print("Original pi:", pi)

This result means that in the long run, about 83.3% of days will be sunny, and 16.7% will be rainy, regardless of the weather on day 1.

#### Visualizing Convergence to Stationary Distribution

Let's see how the probability distribution evolves over time. We'll start with a 100% chance of being Sunny (`[1, 0]`) and see how many steps it takes to get close to `pi`.

In [ ]:
# Start with a sunny day: initial distribution is [1, 0]
initial_dist = np.array([1, 0])

n_steps = 15
prob_history = np.zeros((n_steps, 2))
prob_history[0, :] = initial_dist

current_dist = initial_dist
for i in range(1, n_steps):
    current_dist = current_dist @ P
    prob_history[i, :] = current_dist

# Plot the convergence
plt.figure(figsize=(12, 6))
plt.plot(prob_history[:, 0], label='P(State = Sunny)', marker='o')
plt.plot(prob_history[:, 1], label='P(State = Rainy)', marker='o')

# Plot the stationary distribution as horizontal lines
plt.axhline(y=pi[0], color='blue', linestyle='--', label=f'Stationary Sunny ({pi[0]:.2f})')
plt.axhline(y=pi[1], color='orange', linestyle='--', label=f'Stationary Rainy ({pi[1]:.2f})')

plt.title('Convergence to Stationary Distribution')
plt.xlabel('Number of Days (Steps)')
plt.ylabel('Probability')
plt.legend()
plt.grid(True)
plt.show()

As you can see from the plot, the probabilities converge very quickly to their long-run equilibrium values, usually within 5-10 days.

## Summary & Next Steps

In this notebook, we've covered the fundamentals of discrete-time Markov Chains:
1.  The **Markov Property**: The future depends only on the present.
2.  The **Transition Probability Matrix (TPM)** as the engine of the chain.
3.  How to **simulate** a path of a Markov Chain.
4.  How to calculate **multi-step transition probabilities** using matrix powers.
5.  The concept of a **stationary distribution** and how to calculate it by finding the eigenvector of the TPM.

Next, we will explore **Hidden Markov Models (HMMs)**. What if we can't observe the state directly (e.g., we don't know if it's Sunny or Rainy), but can only see a related output (e.g., we see people carrying umbrellas)? HMMs allow us to model such systems.